# http wrapper for kachaka-api

kachaka-apiライブラリをhttp経由でアクセスできるようにするサンプルです  
```
curl <カチャカのIPアドレス>:26502/kachaka/robot_serial_number
```

In [ ]:
%run setup_demo_env.py

In [ ]:
!pip3 install fastapi uvicorn

In [ ]:
from fastapi import FastAPI, HTTPException
import kachaka_api

app = FastAPI()
kachaka_client = kachaka_api.aio.KachakaApiClient(kachaka_api_server)


@app.on_event("startup")
async def init_channel() -> None:
    await kachaka_client.update_resolver()


def to_dict(response):
    if response.__class__.__module__ != "kachaka_api_pb2":
        return response
    ret = {}
    for line in response.__repr__().split("\n")[:-1]:
        key = line.split(":")[0]
        ret[key] = getattr(response, key)
    return ret


def get_method_or_404(attributes):
    for attr in attributes:
        if hasattr(kachaka_client, attr):
            return getattr(kachaka_client, attr)
    raise HTTPException(status_code=404, detail="Method not found")


@app.get("/kachaka/{item:path}")
async def get(item: str):
    method = get_method_or_404(["get_" + item, item])
    response = await method()
    return to_dict(response)


@app.post("/kachaka/{item:path}")
async def post(item: str, params: dict):
    method = get_method_or_404([item, "set_" + item])
    response = await method(**params)
    return to_dict(response)


In [ ]:
import uvicorn
import asyncio

config = uvicorn.Config(app)
config.host = "0.0.0.0"
config.port = 26502
server = uvicorn.Server(config)
await server.serve()